<a href="https://colab.research.google.com/github/BDonadelli/Codigos-em-financas/blob/main/extrato_BB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://docs.google.com/spreadsheets/d/1PIuPrpaZOWMtOtjbkOHCFNLRdcTUlYj1g_-WDFFt2NE/edit#gid=1490406301

In [ ]:
# # testa se roda pelo colab
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

import gspread
import pandas as pd

In [ ]:
# # se rodar pelo colab
if IN_COLAB  : # vai pegar o ultimo extrato da planilha de extratos e por num dataframe
    # from gspread_dataframe import get_as_dataframe, set_with_dataframe
    from google.colab import auth
    auth.authenticate_user()
    from google.auth import default
    credentials, _ = default()
    gc = gspread.authorize(credentials)
    planilha = gc.open('extrato BB 2023')
    df = pd.DataFrame(planilha.get_worksheet(1).get_all_records())
    df['Data'] = pd.to_datetime(df['Data'], dayfirst=True)
    df['Valor'] = df['Valor'].replace(regex={',': '.'}).astype(float)
    df['Número do documento'] = df['Número do documento'].fillna(' ').astype(str)
    # gc.session.close()
#########################################################################################################
# se rodar via google cloud
else:  # vai pegar o ultimo extrato do computador,  or num dataframe e escrever na planilha de extratos
    from oauth2client.service_account import ServiceAccountCredentials

    scope = ['https://www.googleapis.com/auth/drive','https://www.googleapis.com/auth/spreadsheets']
    credentials = ServiceAccountCredentials.from_json_keyfile_name('destrincha-fatura.json',scope)
    gc = gspread.authorize(credentials)
    planilha = gc.open('extrato BB 2023')
    path='/home/yair/Dropbox/Desktop/Financas/extrato.csv'
    df = pd.read_csv(path,
                      sep=',' , decimal='.' ,
                      header = 0, index_col=False,
                      parse_dates=[0] ,
                      usecols=['Data' , 'Dependencia Origem' , 'Histórico' , 'Data do Balancete' , 'Número do documento' , 'Valor'] ,
                      encoding='iso-8859-1',
                      dayfirst=True
                      )
    aba=str(input("mês ano (mmm YY): "))
    try :
        pagina = planilha.worksheet(aba)
        pagina.clear()
    except :
        pagina = planilha.add_worksheet(title=aba, rows=100, cols=20 , index= 1)
    # ajeita df senão não escreve no gs
    df['Data'] = df['Data'].dt.strftime("%d/%m/%Y")
    df['Dependencia Origem'] = df['Dependencia Origem'].fillna(' ').astype(str)
    df['Data do Balancete'] = df['Data do Balancete'].fillna(' ').astype(str)
    df['Número do documento'] = df['Número do documento'].fillna(' ').astype(str)
    # escreve no gs
    pagina.update([df.columns.values.tolist()] + df.values.tolist())
    df['Data'] = pd.to_datetime(df['Data'], dayfirst=True)
    # gc.session.close()
#########################################################################################################
# df

/home/yair/.local/lib/python3.7/site-packages/gspread/worksheet.py:1049: UserWarning: [Deprecated][in version 6.0.0]: method signature will change to: 'Worksheet.update(value = [[]], range_name=)' arguments 'range_name' and 'values' will swap, values will be mandatory of type: 'list(list(...))'
  msg_deprecated="method signature will change to: 'Worksheet.update(value = [[]], range_name=)'"


In [ ]:
def limpa(espacos) :
  return " ".join(espacos.split())

def que_receita(string) :
  ''' associa a 'string' à chave do dicionário 'Despesas' ou 'Receitas' '''
  for t in receitas.keys() :
    for casos in receitas[t]:
      if casos in string:
        return t,casos
  else: return 'nao achei '+'"'+string+'"',None

def que_despesa(string) :
  ''' associa a 'string' à chave do dicionário 'Despesas' ou 'Receitas' '''
  for t in Despesas.keys() :
    for casos in Despesas[t]:
      if casos in string:
        return t,casos
  else: return 'nao achei '+'"'+string+'"',None

RF      = ['Atacadao' , 'Raizen' , 'AES' , 'Eldorado' , 'Tesouro Direto']
#-CRA ATACADAO, -DEB AES TIETE,  -DEB RAIZEN,  -CRA ELDORADO, Tesouro Direto-Juros
RV      = ['BBAS3' , 'BBDC3' , 'PSSA3' , 'SAPR11' , 'ITSA4' , 'BOAC34']

In [ ]:
if IN_COLAB  :
    import requests
    url = str(input('pegar url no github (https://github.com/BDonadelli/Utilitarios/blob/main/BBdict.json): '))
    f = requests.get(url).json()
    receitas = f[0]
    Despesas = f[1]
else :
    from BBdict import *

In [ ]:
tipo = []
onde=[]
classe = []
i=0
while i < len(df):
  # pula as linhas com 'saldo'
  if 'saldo' in df['Histórico'].iloc[i].replace(" ", "").lower() :
    tipo.append('-')
    onde.append('-')
    classe.append('-')
    i=i+1

  elif df['Valor'].iloc[i] > 0  :
    #print(df['Histórico'].iloc[i]+'----'+que_receita(df['Histórico'].iloc[i]))
    tipo.append('C')
    aux1,aux2 = que_receita(df['Histórico'].iloc[i])
    if aux1 == 'Cupon RF' :
      if 'Tesouro Direto' in aux2 : aux2 = 'TD'
      else :
        for j in RF :
          if j in df['Histórico'].iloc[i] or j.upper() in df['Histórico'].iloc[i]: aux2=j
      aux1 =aux1+' '+aux2
    if aux1 == 'JCP' or aux1 == 'Dividendos':
      for j in RV:
        if j in df['Histórico'].iloc[i]:
          aux1 =aux1+' '+j
    onde.append(aux1)
    classe.append('Recebido')
    i=i+1

  elif df['Valor'].iloc[i]< 0 :
    #print(df['Histórico'].iloc[i]+' ---- '+que_despesa(df['Histórico'].iloc[i]))
    tipo.append('D')
    aux,aux2 = que_despesa(df['Histórico'].iloc[i])
    if aux == 'Med/Dent/Psic':
      if aux2 == 'DENISE' : aux2 = 'Denise'
    if aux == 'Educação':
      aux2 = 'Singular'
    if aux == 'Taxas':
      aux2='Imposto(ñ carro/casa)'
    if aux == 'Cheque':
        if df['Valor'].iloc[i] == 2500.00 : aux2 = "Hosp. São Luiz"
        if df['Valor'].iloc[i] == -236.21 : aux2 = "Singular"
    if aux == 'Sacolão': aux2 = 'Rogério'
    if aux == 'Plano de saúde': aux2 = 'Unimed'
    if aux == 'Lazer': aux2 = 'Afpesp'
    if aux == 'Gás': aux2 = 'Ultragaz'
    if aux == 'Luz': aux2 = 'Eletropaulo'
    if aux == 'Condominio' : aux2 = aux
    if aux == 'Lighia' and 'PORTO SEGURO CARTOES' in aux2: aux2='Lighia'
    onde.append(aux2)
    classe.append(aux)
    i=i+1

df['tipo'] = tipo
df['onde'] = onde
df['classe']=classe


In [ ]:
creditos = df[df.tipo=='C'].sort_values(by=['Data'])
creditos

,Data,Dependencia Origem,Histórico,Data do Balancete,Número do documento,Valor,tipo,onde,classe
1,2023-11-01,,Recebimento de Proventos - FUNDACAO UNIVERSIDA...,,263521,10698.74,C,Salário,Recebido
5,2023-11-06,7068-8,Transferência recebida - 06/11 21:19 LIGHIA B ...,,527068000014538,912.52,C,Lighia,Recebido
9,2023-11-08,7068-8,Transferência recebida - 08/11 20:08 LIGHIA B ...,,527068000014538,283.50,C,Lighia,Recebido
10,2023-11-08,,Proventos - Pag Rendimento LVBI11 08/11/2023,,103120800027974,69.60,C,FII,Recebido


In [ ]:
debitos = df[df.tipo=='D'].sort_values(by=['Data'])
debitos

,Data,Dependencia Origem,Histórico,Data do Balancete,Número do documento,Valor,tipo,onde,classe
2,2023-11-01,,TED - 077 0001 11572314893 JAIR DONADELLI JU,,110101,-8000.00,D,JAIR DONADELLI J,Eu
3,2023-11-01,1981-X,Cheque Compensado,,850365,-236.21,D,Singular,Cheque
4,2023-11-01,,Pagamento Fatura de Gás - ULTRAGAZ - DEMAIS RE...,,3128,-194.19,D,Ultragaz,Gás
6,2023-11-06,,Pagamento de Boleto - CONDOMINIO EDIFICIO C DE...,,110601,-1227.86,D,Condominio,Condominio
7,2023-11-06,,Pix - Enviado - 06/11 17:07 Rogerio Yoshikazu ...,,110602,-60.00,D,Rogério,Sacolão
8,2023-11-06,,Net Serviços de Comunicaç - NET SERVIçOS,,41843,-225.04,D,Net,TV/Internet
11,2023-11-08,,Pix - Enviado - 08/11 16:50 Adeilzo Jose Dos S...,,110801,-283.50,D,Adeilzo,Lighia
12,2023-11-08,,Pix - Enviado - 08/11 20:48 Rogerio Yoshikazu ...,,110802,-29.86,D,Rogério,Sacolão
13,2023-11-09,,Taxa Compra/Venda Ações,,103130800031483,-0.02,D,Imposto(ñ carro/casa),Taxas
14,2023-11-09,,Compra de Ações,,103130800031485,-420.66,D,Compra de Ações,Investimento


In [ ]:
final = pd.concat([creditos,debitos])
final['Valor']=final['Valor'].apply(abs)
final['Data'] = final['Data'].dt.strftime('%d/%m/%Y')

In [ ]:
resumo = final[['Histórico','onde','Data','classe','Valor','tipo']].copy()
resumo

,Histórico,onde,Data,classe,Valor,tipo
1,Recebimento de Proventos - FUNDACAO UNIVERSIDA...,Salário,01/11/2023,Recebido,10698.74,C
5,Transferência recebida - 06/11 21:19 LIGHIA B ...,Lighia,06/11/2023,Recebido,912.52,C
9,Transferência recebida - 08/11 20:08 LIGHIA B ...,Lighia,08/11/2023,Recebido,283.50,C
10,Proventos - Pag Rendimento LVBI11 08/11/2023,FII,08/11/2023,Recebido,69.60,C
2,TED - 077 0001 11572314893 JAIR DONADELLI JU,JAIR DONADELLI J,01/11/2023,Eu,8000.00,D
3,Cheque Compensado,Singular,01/11/2023,Cheque,236.21,D
4,Pagamento Fatura de Gás - ULTRAGAZ - DEMAIS RE...,Ultragaz,01/11/2023,Gás,194.19,D
6,Pagamento de Boleto - CONDOMINIO EDIFICIO C DE...,Condominio,06/11/2023,Condominio,1227.86,D
7,Pix - Enviado - 06/11 17:07 Rogerio Yoshikazu ...,Rogério,06/11/2023,Sacolão,60.00,D
8,Net Serviços de Comunicaç - NET SERVIçOS,Net,06/11/2023,TV/Internet,225.04,D


In [ ]:
pagina = gc.open('Planilha para Controle Financeiro 2021 em diante').worksheet('extratoBB')

In [ ]:
pagina.clear()

{'spreadsheetId': '1chQdqBEPA_E_5eBWSP2j0KvEwBWZxzjEPJv8oyTSNPk',
 'clearedRange': 'extratoBB!A1:Z1000'}

In [ ]:
pagina.update([resumo.columns.values.tolist()] + resumo.values.tolist())

/home/yair/.local/lib/python3.7/site-packages/gspread/worksheet.py:1049: UserWarning: [Deprecated][in version 6.0.0]: method signature will change to: 'Worksheet.update(value = [[]], range_name=)' arguments 'range_name' and 'values' will swap, values will be mandatory of type: 'list(list(...))'
  msg_deprecated="method signature will change to: 'Worksheet.update(value = [[]], range_name=)'"


{'spreadsheetId': '1chQdqBEPA_E_5eBWSP2j0KvEwBWZxzjEPJv8oyTSNPk',
 'updatedRange': 'extratoBB!A1:F19',
 'updatedRows': 19,
 'updatedColumns': 6,
 'updatedCells': 114}

In [ ]:
resumo.groupby('onde').sum()

,Valor
onde,
Adeilzo,283.50
Afpesp,42.50
Compra de Ações,420.66
Condominio,1227.86
Débito de Ações,22.73
FII,69.60
Imposto(ñ carro/casa),0.12
JAIR DONADELLI J,8000.00
Lighia,2050.97
